# Demos: Lecture 7

## Demo 1: expectation value from samples

In [18]:
import pennylane as qml
import numpy as np

dev = qml.device('default.qubit', wires=1, shots=1000)
  
@qml.qnode(dev)
def measure_z():
    qml.RX(2*np.pi/3, wires=0)    
    return qml.sample(qml.PauliZ(0))

## Demo 2: multi-qubit expectation values

In [38]:
dev = qml.device('default.qubit', wires=2)
  
@qml.qnode(dev)
def measure_z():
    qml.PauliX(wires=1)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))